# 필요한 거 import

In [7]:
# pip install selenium
# pip install webdriver_manager
import os
import pandas as pd

### 만약에 코드 안돌아가면 이 위에꺼 주석처리 해제하고 한 번 돌려주심 됩니다.

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import pandas as pd
import credentials  # credentials.py 파일에서 사용자 이름과 비밀번호를 가져옴


### init driver

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--headless') ##이거 주석처리 지우면 브라우저 안뜸 (걍 뒤에서 크롤링)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

### login

##### credentials.py 에서 자기 학번 이름 넣으면 됩니당.


In [10]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "loginid"))).send_keys(credentials.USERNAME)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "loginpw"))).send_keys(credentials.PASSWORD)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "searchbtn"))).click()
time.sleep(5)

### 학교 링크 얻는 함수 

In [11]:
def get_school_links(driver, country_code):
    base_url = 'https://oia.yonsei.ac.kr/partner/expReport.asp'
    driver.get(f'{base_url}?yn=Y&country_code={country_code}&univ=')
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = [urljoin(base_url, a['href']) for a in soup.select('div.schWrap tbody td.left a')]
    return links

### 교환 후기 크롤링해오는 함수

In [12]:
from selenium.common.exceptions import TimeoutException



def scrape_reviews(driver, start_url, country_code, collected_review_links):
    reviews = []
    try:
        driver.get(start_url)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.wp-pagenavi')))
    except TimeoutException:
        print(f"Timeout or page not found for {start_url}")
        return reviews  # 해당 학교 페이지가 로드되지 않을 때 빈 리스트를 반환

    current_page = 1  # 시작 페이지 번호 초기화

    # 학교 이름 추출
    school_name_html = driver.page_source
    school_soup = BeautifulSoup(school_name_html, 'html.parser')
    school_name = school_soup.select_one('h4.tit').text.strip() if school_soup.select_one('h4.tit') else 'Unknown School'

    no_review_element = school_soup.select_one('div.no-reviews')  # 예를 들어 리뷰가 없을 때 표시되는 요소
    if no_review_element:
        print(f"No reviews available for {start_url}")
        return reviews  # 리뷰가 없는 경우 빈 리스트 반환

    while True:
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.schWrap tbody tr')))
            page_html = driver.page_source
            page_soup = BeautifulSoup(page_html, 'html.parser')

        except TimeoutException:
            print("No review entries found on the page, moving to next school.")
            break  # 리뷰 목록이 로드되지 않으면 루프 탈출

        # 현재 페이지 내의 리뷰 항목 처리
        review_entries = page_soup.select('div.schWrap tbody tr')
        for entry in review_entries:
            review_link_tag = entry.select_one('td.left a')
            if review_link_tag:
                review_link = urljoin(start_url, review_link_tag['href'])
                if review_link not in collected_review_links:
                    collected_review_links.add(review_link)
                    driver.get(review_link)
                    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.exp_txt span')))
                    detail_html = driver.page_source
                    detail_soup = BeautifulSoup(detail_html, 'html.parser')
                    review_title = detail_soup.select_one('li.title_view h4').text.strip() if detail_soup.select_one('li.title_view h4') else 'No Title'
                    exp_txt_blocks = detail_soup.select('div.exp_txt span')
                    review_texts = [span.text.strip().replace('\n', ' ') for span in exp_txt_blocks]
                    reviews.append({
                        'Country Code': country_code,
                        'School Name': school_name,
                        'Review Link': review_link,
                        'Review Title': review_title,
                        'Review Content': " ".join(review_texts)
                    })
                    driver.back()

        # 다음 페이지 링크 처리
        if current_page % 5 == 0:  # 매 5페이지마다 nextpostslink 사용
            next_link = page_soup.select_one('a.nextpostslink')
            if next_link and next_link['href'] != '#':
                new_url = urljoin(start_url, next_link['href'])
                print(f"Moving to next page group at {new_url}")
                driver.get(new_url)
                current_page += 1
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.wp-pagenavi')))
            else:
                print("No more pages to process.")
                break
        else:
            page_links = page_soup.select('a.page.smaller')
            next_page_found = False
            for link in page_links:
                if int(link.text) == current_page + 1:
                    new_url = urljoin(start_url, link['href'])
                    print(f"Moving to page {link.text} at {new_url}")
                    driver.get(new_url)
                    current_page += 1
                    next_page_found = True
                    break
            if not next_page_found:
                print("No more pages to process.")
                break

    return reviews

def scrape_schools(driver, country_code, school_links):
    collected_review_links = set()
    file_path = f'reviewfiles/{country_code}_exchange_student_reviews.csv'
    
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        collected_review_links = set(df['Review Link'])

    all_reviews = []
    for link in school_links:
        reviews = scrape_reviews(driver, link, country_code, collected_review_links)
        if reviews:  # 리뷰가 수집된 경우에만 추가
            all_reviews.extend(reviews)

    if all_reviews:
        df = pd.DataFrame(all_reviews)
        df.to_csv(file_path, mode='a', index=False, header=not os.path.exists(file_path))
        print(f"New reviews have been appended to '{file_path}'.")
    else:
        print("No new reviews were collected.")

    driver.quit()


# 예시 사용법
country_code = 'UK'
school_links = get_school_links(driver, country_code)


## 75번 대학부터 제대로 크롤링되기 시작 ^^^ 
# 학교 리스트를 10개씩 끊어서 크롤링
for i in range(1, len(school_links),213):
    scrape_schools(driver, country_code, school_links[i:i+213])


### 각 함수들 실행할 메인

##### ** 수정필요 : 국가 코드 수정해주셔야 됨
###### 지금 나고 있는 에러는 무시해주시고, 위에서부터 차례차례 돌리면 에러 안나유~